**Andrew Kittredge**<br>
**GTECH 731**<br>
**Assignment 6**

# Batch raster clipping using `arcpy`

Import packages, set workspace to current project geodatabase:

In [65]:
import os, fnmatch, arcpy
aprx = arcpy.mp.ArcGISProject("Current")
arcpy.env.workspace = aprx.defaultGeodatabase

Print the rasters present in a directory. These are summer Landsat scenes over New York City from 1984 - 1988.

In [81]:
rasterPath = r'C:\Users\AndysWorld\Desktop\Landsat\1985+'

for root, dirs, files in os.walk(rasterPath, topdown=True):
    for name in fnmatch.filter(files, "*.tif"):
        print(os.path.join(root, os.path.splitext(name)[0]))

C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_013032_19840610_20200918_02_T1\LT05_L2SP_013032_19840610_20200918_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_013032_19880808_20200917_02_T1\LT05_L2SP_013032_19880808_20200917_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_013032_19890726_20200916_02_T1\LT05_L2SP_013032_19890726_20200916_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_014032_19840703_20200918_02_T1\LT05_L2SP_014032_19840703_20200918_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_014032_19840719_20200918_02_T2\LT05_L2SP_014032_19840719_20200918_02_T2_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_014032_19860810_20200917_02_T1\LT05_L2SP_014032_19860810_20200917_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_014032_19870728_20201014_02_T1\LT05_L2SP_014032_19870728_20201014_02_T1_ST_B6
C:\Users\AndysWorld\Desktop\Landsat\1985+\LT05_L2SP_014032_19880730_20200917_02_T1\LT05_L2

`arcpy.management.Clip()` is used to clip the rasters using a shapefile of New York City county land boundaries. A for loop is used to create a save file path and file name for each raster. 

In [82]:
nycLandMask = r'Z:\Windows\731-ak-assignment-6\nyc_counties\nyc_counties.shp'
outFolder = r'Z:\Windows\731-ak-assignment-6\clipped_rasters'

for root, dirs, files in os.walk(rasterPath, topdown=True):
    for name in fnmatch.filter(files, "*.tif"):
        outRaster = os.path.join(outFolder, name) + '.tif'
        # arcpy.management.Clip(in_raster, rectangle, out_raster, {in_template_dataset}, {nodata_value}, {clipping_geometry}, {maintain_clipping_extent})
        arcpy.management.Clip(os.path.join(root, name),'#',outRaster, nycLandMask, '#', 'ClippingGeometry')

The resulting clipped rasters:

In [84]:
for root, dirs, files in os.walk(outFolder, topdown=True):
    for name in fnmatch.filter(files, "*.tif"):
        print(os.path.join(root, os.path.splitext(name)[0]))

Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19870728_20201014_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19840703_20200918_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19860810_20200917_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_013032_19890726_20200916_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_013032_19840610_20200918_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_013032_19880808_20200917_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19880815_20200917_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19880730_20200917_02_T1_ST_B6.TIF
Z:\Windows\731-ak-assignment-6\clipped_rasters\LT05_L2SP_014032_19840719_20200918_02_T2_ST_B6.TIF


Unlike the example provided in lecture, using `arcpy.ListRasters()` as an input to `arcpy.management.Clip()` as it does not work with rasters loaded into layer files:

In [85]:
for root, dirs, files in os.walk(rootPath, topdown=True):
    for name in fnmatch.filter(files, "*.tif"):
        arcpy.management.MakeRasterLayer(os.path.join(root, name), os.path.splitext(name)[0],"#", "nyc_counties/nyc_counties.shp")
        
arcpy.ListRasters()

[]

Condensed script for toolbox:

In [ ]:
import os, fnmatch, arcpy
def batchRasterClip(inputFolder, outputFolder, clipShp):
    aprx = arcpy.mp.ArcGISProject("Current")
    arcpy.env.workspace = aprx.defaultGeodatabase
    
    for root, dirs, files in os.walk(inputFolder, topdown=True):
        for name in fnmatch.filter(files, "*.tif"):
            outRaster = os.path.join(outputFolder, name) + '.tif'
            # arcpy.management.Clip(in_raster, rectangle, out_raster, {in_template_dataset}, {nodata_value}, {clipping_geometry}, {maintain_clipping_extent})
            arcpy.management.Clip(os.path.join(root, name),'#',outRaster, clipShp, '#', 'ClippingGeometry')
    